<a href="https://colab.research.google.com/github/CUSPADS2022IBX/IBXRidership/blob/main/Turnstile%20Data%20Processing/Turnstile_Auditing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Source: http://web.mta.info/developers/turnstile.html

Example:

The data below shows the entry/exit register values for one turnstile at control area (A002) from 09/27/14 at 00:00 hours to 09/29/14 at 00:00 hours

Schema Example:
C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,00:00:00,REGULAR,0004800073,0001629137,

Data cleaning and processing resources used:

1)https://medium.com/qri-io/taming-the-mtas-unruly-turnstile-data-c945f5f96ba0

2)https://toddwschneider.com/dashboards/nyc-subway-turnstiles/#notes


In [84]:
!pip install pyspark
!pip install --upgrade xlrd

In [85]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window
from pyspark import SparkFiles

sc = pyspark.SparkContext.getOrCreate()
spark = SparkSession(sc)

#The start date has to match the date of the first URL in the MTA turnstile data, otherwise the URL pull will not work. Data is reported every Saturday.
#NOTE: On November 20, 2021 MTA changed their normal turnstile count periods from [12AM, 4AM, 8AM, 12PM, 4PM, 8PM] to [3AM, 7AM, 11AM, 3PM, 7PM, 11PM]
start_date = date(2022,3, 5)
end_date = date(2022, 4, 23)

#Create a list of dates for the date range requested
date_range = list(pd.date_range(start_date, end_date, freq='7D').strftime("%y%m%d"))

#MTA tunrstile schema, 'EXITS' kepts giving nulls when imported as IntegerType
mta_turnstile_schema = T.StructType([
  T.StructField('C/A', T.StringType(), True),
  T.StructField('UNIT', T.StringType(), True),
  T.StructField('SCP', T.StringType(), True),
  T.StructField('STATION', T.StringType(), True),
  T.StructField('LINENAME', T.StringType(), True),
  T.StructField('DIVISION', T.StringType(), True),
  T.StructField('DATE', T.StringType(), True),
  T.StructField('TIME', T.StringType(), True),
  T.StructField('DESC', T.StringType(), True),
  T.StructField('ENTRIES', T.IntegerType(), True),
  T.StructField('EXITS', T.FloatType(), True),
  ])

#Create empty dataframe with previous scheme
bigdf = spark.createDataFrame([], mta_turnstile_schema)

#Download each .txt file on to Spark job node and load into Spark DataFrame and union onto Empty DataFrame we created
for date_string in date_range:
  url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt'.format(date_string)
  spark.sparkContext.addFile(url)
  df = spark.read.csv(SparkFiles.get('turnstile_{}.txt'.format(date_string)), mta_turnstile_schema, header=True)
  bigdf = bigdf.union(df)

#Change 'EXITS' column data type to IntegerType and concate 'DATE' and 'TIME' columns and cast to datetime
bigdf = bigdf.withColumn('EXITS',bigdf.EXITS.cast(T.IntegerType()))\
             .withColumn('timestamp',
                         F.unix_timestamp(F.concat(bigdf.DATE,bigdf.TIME),'MM/dd/yyyyHH:mm:ss').cast('timestamp'))
             
#Create columns to represent unique observation id, and unique turnstile id for data processing             
bigdf = bigdf.withColumn('unit_division', F.concat(bigdf.UNIT,bigdf.DIVISION))\
             .withColumn('unit_id', F.concat(bigdf['C/A'],bigdf.UNIT,bigdf.SCP))

In [86]:
bigdf.show(5)

+----+----+--------+-------+--------+--------+----------+--------+-------+-------+-------+-------------------+-------------+----------------+
| C/A|UNIT|     SCP|STATION|LINENAME|DIVISION|      DATE|    TIME|   DESC|ENTRIES|  EXITS|          timestamp|unit_division|         unit_id|
+----+----+--------+-------+--------+--------+----------+--------+-------+-------+-------+-------------------+-------------+----------------+
|A002|R051|02-00-00|  59 ST| NQR456W|     BMT|02/26/2022|03:00:00|REGULAR|7689737|2671257|2022-02-26 03:00:00|      R051BMT|A002R05102-00-00|
|A002|R051|02-00-00|  59 ST| NQR456W|     BMT|02/26/2022|07:00:00|REGULAR|7689741|2671278|2022-02-26 07:00:00|      R051BMT|A002R05102-00-00|
|A002|R051|02-00-00|  59 ST| NQR456W|     BMT|02/26/2022|11:00:00|REGULAR|7689758|2671380|2022-02-26 11:00:00|      R051BMT|A002R05102-00-00|
|A002|R051|02-00-00|  59 ST| NQR456W|     BMT|02/26/2022|15:00:00|REGULAR|7689791|2671465|2022-02-26 15:00:00|      R051BMT|A002R05102-00-00|
|A002|

In [87]:
print(bigdf.count())
bigdf.describe(['ENTRIES', 'EXITS']).show()

1684150
+-------+--------------------+--------------------+
|summary|             ENTRIES|               EXITS|
+-------+--------------------+--------------------+
|  count|             1684150|             1684150|
|   mean|  4.27020266821542E7|3.4312966725517325E7|
| stddev|2.2301099300602618E8|  1.96194359027336E8|
|    min|                   0|                   0|
|    max|          2147319334|          2122443392|
+-------+--------------------+--------------------+



In [88]:
#Use utility function window to partition by turnstile and order by timestamp
window = Window.partitionBy('unit_id').orderBy('timestamp')

#Use previous window to find the 'net_entries' and 'net_exits'. Remove all entries that are above 10000, because
#turnstiles act as odometers, and when turnstile reaches end it resets creating a large value. 10000 is a good cutoff.
#Also drop first rows of each turnstile data, because .lag function creates None for first row.
net_entry_exit = bigdf.withColumn('net_entries', F.abs(F.col('ENTRIES') - F.lag(F.col('ENTRIES'), 1).over(window)))\
                      .withColumn('net_exits', F.abs(F.col('EXITS') - F.lag(F.col('EXITS'), 1).over(window))).dropna()

In [89]:
print(net_entry_exit.count())
net_entry_exit.describe(['net_entries', 'net_exits']).show()

1679109
+-------+------------------+------------------+
|summary|       net_entries|         net_exits|
+-------+------------------+------------------+
|  count|           1679109|           1679109|
|   mean|3636.3177625752705| 3958.484421201959|
| stddev|1866207.4947073061|2090951.0729941803|
|    min|                 0|                 0|
|    max|        1278711326|        1871701216|
+-------+------------------+------------------+



In [90]:
net_entry_exit = net_entry_exit.filter((F.col('net_entries')<10000) & (F.col('net_exits')<10000))

In [91]:
print(net_entry_exit.count())
net_entry_exit.describe(['net_entries', 'net_exits']).show()

1678901
+-------+-----------------+------------------+
|summary|      net_entries|         net_exits|
+-------+-----------------+------------------+
|  count|          1678901|           1678901|
|   mean|70.30216433250085| 89.40866793217707|
| stddev|  104.14214187233|142.35420633867332|
|    min|                0|                 0|
|    max|             9834|              8934|
+-------+-----------------+------------------+



In [92]:
net_entry_exit.show(5)

+----+----+--------+-------+--------+--------+----------+--------+-------+-------+-------+-------------------+-------------+----------------+-----------+---------+
| C/A|UNIT|     SCP|STATION|LINENAME|DIVISION|      DATE|    TIME|   DESC|ENTRIES|  EXITS|          timestamp|unit_division|         unit_id|net_entries|net_exits|
+----+----+--------+-------+--------+--------+----------+--------+-------+-------+-------+-------------------+-------------+----------------+-----------+---------+
|A002|R051|02-00-01|  59 ST| NQR456W|     BMT|02/26/2022|07:00:00|REGULAR|6803071|1576712|2022-02-26 07:00:00|      R051BMT|A002R05102-00-01|          2|       10|
|A002|R051|02-00-01|  59 ST| NQR456W|     BMT|02/26/2022|11:00:00|REGULAR|6803085|1576759|2022-02-26 11:00:00|      R051BMT|A002R05102-00-01|         14|       47|
|A002|R051|02-00-01|  59 ST| NQR456W|     BMT|02/26/2022|15:00:00|REGULAR|6803131|1576797|2022-02-26 15:00:00|      R051BMT|A002R05102-00-01|         46|       38|
|A002|R051|02-00

In [93]:
#Upload Remote_complex_lookup table and create key table for unit_division join
#Manually checked if complex_id was correct (google sheets for reference: https://docs.google.com/spreadsheets/d/1kMmoqzq3uWM5J8Esrzi1DPEBrdezzVtQ1Rv5ZAIsEfk/edit?usp=sharing)
remote_complex_url = 'https://raw.githubusercontent.com/qri-io/data-stories-scripts/master/nyc-turnstile-counts/lookup/remote_complex_lookup.csv'
remote_complex = pd.read_csv(remote_complex_url).sort_values('station')
remote_complex['complex_id'] = remote_complex['complex_id'].astype('Int64').astype('str')
remote_complex['unit_division'] = remote_complex['remote ']+remote_complex['division']
remote_complex = pd.DataFrame(remote_complex.groupby(['unit_division', 'complex_id']).size()).reset_index()
remote_complex_spark = spark.createDataFrame(remote_complex)

print(remote_complex.count())
remote_complex_spark.show(5)

unit_division    506
complex_id       506
0                506
dtype: int64
+-------------+----------+---+
|unit_division|complex_id|  0|
+-------------+----------+---+
|      R001BMT|       635|  2|
|      R001IRT|       635|  1|
|      R002BMT|       628|  3|
|      R003BMT|        86|  1|
|      R004BMT|        85|  1|
+-------------+----------+---+
only showing top 5 rows



In [94]:
#join to the entry_exit_df to create unique complex_Id column to aggregate on
complex_id_df = net_entry_exit.join(remote_complex_spark, net_entry_exit.unit_division==remote_complex_spark.unit_division, how='left')\
                              .select('complex_id','net_entries','net_exits', 'timestamp').sort(F.col('complex_id')).dropna()

In [95]:
complex_id_df.show(5)

+----------+-----------+---------+-------------------+
|complex_id|net_entries|net_exits|          timestamp|
+----------+-----------+---------+-------------------+
|         1|         26|       49|2022-02-26 07:00:00|
|         1|         12|      200|2022-02-27 03:00:00|
|         1|        123|      194|2022-02-26 11:00:00|
|         1|        146|      310|2022-02-26 15:00:00|
|         1|        122|      422|2022-02-26 19:00:00|
+----------+-----------+---------+-------------------+
only showing top 5 rows



In [96]:
print(complex_id_df.count())
complex_id_df.describe(['net_entries', 'net_exits']).show()

1669809
+-------+------------------+------------------+
|summary|       net_entries|         net_exits|
+-------+------------------+------------------+
|  count|           1669809|           1669809|
|   mean| 70.50102137430089| 89.51339344799315|
| stddev|104.30069926047543|142.42225900403477|
|    min|                 0|                 0|
|    max|              9834|              8934|
+-------+------------------+------------------+



In [97]:
#Create new column for 'Day' 
DOW_df = complex_id_df.withColumn('Day', F.date_trunc('day', F.col('timestamp')))

In [98]:
DOW_df.show(50)

+----------+-----------+---------+-------------------+-------------------+
|complex_id|net_entries|net_exits|          timestamp|                Day|
+----------+-----------+---------+-------------------+-------------------+
|         1|         20|       18|2022-02-26 07:00:00|2022-02-26 00:00:00|
|         1|        115|       87|2022-03-06 15:00:00|2022-03-06 00:00:00|
|         1|        111|       73|2022-02-26 11:00:00|2022-02-26 00:00:00|
|         1|        132|      132|2022-02-26 15:00:00|2022-02-26 00:00:00|
|         1|        116|      248|2022-02-26 19:00:00|2022-02-26 00:00:00|
|         1|         61|      164|2022-02-26 23:00:00|2022-02-26 00:00:00|
|         1|         11|      100|2022-02-27 03:00:00|2022-02-27 00:00:00|
|         1|         15|       17|2022-02-27 07:00:00|2022-02-27 00:00:00|
|         1|         88|       54|2022-02-27 11:00:00|2022-02-27 00:00:00|
|         1|         85|      101|2022-02-27 15:00:00|2022-02-27 00:00:00|
|         1|         70| 

In [99]:
#Aggregate on complex_id, timestamp and unit_division to get all entries and exits for each station
complex_id_DAY_df = DOW_df.groupBy('Day','complex_id').sum('net_entries','net_exits').withColumnRenamed('sum(net_entries)', 'entries')\
                                                                                                         .withColumnRenamed('sum(net_exits)', 'exits')
complex_id_DAY_df.show(5)

+-------------------+----------+-------+-----+
|                Day|complex_id|entries|exits|
+-------------------+----------+-------+-----+
|2022-03-02 00:00:00|         9|  12475|15462|
|2022-04-20 00:00:00|        10|   9772|14307|
|2022-03-06 00:00:00|       607|  20526|31707|
|2022-03-12 00:00:00|        14|   5139| 5907|
|2022-04-08 00:00:00|        14|   6878| 5728|
+-------------------+----------+-------+-----+
only showing top 5 rows



In [100]:
complex_id_DAY_df.describe(['entries', 'exits']).show()

+-------+-----------------+------------------+
|summary|          entries|             exits|
+-------+-----------------+------------------+
|  count|            23908|             23908|
|   mean|4924.010373096871| 6251.893508449055|
| stddev|8671.154917856256|11007.321474012007|
|    min|                0|                 0|
|    max|           169204|            173370|
+-------+-----------------+------------------+



In [101]:
#Create new column for 'DOW' (day of week) to aggregate by weekends and weekdays.
DAY_df = complex_id_DAY_df.withColumn('DOW', F.when((F.dayofweek(F.col('Day'))<7) & (F.dayofweek(F.col('Day'))>1),'weekday')\
                                                  .when((F.dayofweek(F.col('Day'))==7) | (F.dayofweek(F.col('Day'))==1),'weekend'))

DAY_df.show(5)

+-------------------+----------+-------+-----+-------+
|                Day|complex_id|entries|exits|    DOW|
+-------------------+----------+-------+-----+-------+
|2022-03-02 00:00:00|         9|  12475|15462|weekday|
|2022-04-20 00:00:00|        10|   9772|14307|weekday|
|2022-03-06 00:00:00|       607|  20526|31707|weekend|
|2022-03-12 00:00:00|        14|   5139| 5907|weekend|
|2022-04-08 00:00:00|        14|   6878| 5728|weekday|
+-------------------+----------+-------+-----+-------+
only showing top 5 rows



In [102]:
#Find average ridership for weekdays and weekends
DAY_avg_df = DAY_df.groupBy('DOW','complex_id').mean('entries','exits').sort(F.col('complex_ID'))

DAY_avg_df.show(5)

+-------+----------+------------+----------+
|    DOW|complex_id|avg(entries)|avg(exits)|
+-------+----------+------------+----------+
|weekend|         1|   3481.4375| 5802.6875|
|weekday|         1|     6937.75|   9643.75|
|weekday|        10|    8465.975| 12732.575|
|weekend|        10|     7251.75|10609.5625|
|weekday|       100|      1212.2|   1950.65|
+-------+----------+------------+----------+
only showing top 5 rows



In [103]:
print(DAY_avg_df.count())
DAY_avg_df.describe(['avg(entries)', 'avg(exits)']).show()

854
+-------+-----------------+------------------+
|summary|     avg(entries)|        avg(exits)|
+-------+-----------------+------------------+
|  count|              854|               854|
|   mean|4332.989199518604| 5662.106111761641|
| stddev|7864.438814557452|10059.578386482686|
|    min|           0.0625|               0.0|
|    max|        147316.35|        151180.825|
+-------+-----------------+------------------+



In [104]:
weekend_avg = DAY_avg_df.filter((DAY_avg_df.DOW == 'weekend')).toPandas()
weekday_avg = DAY_avg_df.filter((DAY_avg_df.DOW == 'weekday')).toPandas()

In [105]:
weekday_avg['avg(entries)'].sum()

2438219.963888889

In [106]:
weekend_avg['avg(entries)'].sum()

1262152.8125

In [59]:
weekend_avg.loc[:425].to_csv('weekend_avg.csv')
weekday_avg.loc[:425].to_csv('weekday_avg.csv')

In [ ]:
stations_url = 'http://web.mta.info/developers/data/nyct/subway/Stations.csv'

stations = pd.read_csv(stations_url).sort_values('Stop Name')

#print(stations.groupby(['Complex ID','Stop Name','GTFS Stop ID','Division']).size().reset_index().rename(columns={0:'count'}))
stations['Stop Name'] =  stations['Stop Name'].str.upper()
#stations['Complex ID', 'GTFS Stop ID','Stop Name'].value_counts()

stations.to_csv('stations_list.csv')

     Complex ID                 Stop Name GTFS Stop ID Division  count
0             1      Astoria-Ditmars Blvd          R01      BMT      1
1             2              Astoria Blvd          R03      BMT      1
2             3                     30 Av          R04      BMT      1
3             4                  Broadway          R05      BMT      1
4             5                     36 Av          R06      BMT      1
..          ...                       ...          ...      ...    ...
491         630        Myrtle-Wyckoff Avs          M08      BMT      1
492         635               South Ferry          142      IRT      1
493         635  Whitehall St-South Ferry          R27      BMT      1
494         636          Jay St-MetroTech          A41      IND      1
495         636          Jay St-MetroTech          R29      BMT      1

[496 rows x 5 columns]


In [ ]:
stations_list = stations.groupby(['Complex ID','Stop Name','GTFS Stop ID', 'Division']).size().reset_index().rename(columns={0:'count'})
stations_list.to_csv('stations_list.csv')

In [ ]:
stop_name = list(stations[['Stop Name','GTFS Stop ID','Complex ID']].unique())
stop_name

In [ ]:
turnstile_key_url = 'http://web.mta.info/developers/resources/nyct/turnstile/Remote-Booth-Station.xls'

turnstile_key = pd.read_excel(turnstile_key_url)
pd.set_option('display.max_rows', turnstile_key.shape[0]+1)

turnstile_key.head()
print(turnstile_key.groupby(['Remote','Station']).size().reset_index().rename(columns={0:'count'}))